In [1]:
import argparse
import os

import torch
import torch.nn as nn


from redunet import *
import evaluate
import load as L
import functional as F
import utils
import plot

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--data', type=str, required=True, help='choice of dataset')
parser.add_argument('--arch', type=str, required=True, help='choice of architecture')
parser.add_argument('--samples', type=int, required=True, help="number of samples per update")
parser.add_argument('--tail', type=str, default='', help='extra information to add to folder name')
parser.add_argument('--save_dir', type=str, default='./saved_models/', help='base directory for saving.')
parser.add_argument('--data_dir', type=str, default='./data/', help='base directory for saving.')

_StoreAction(option_strings=['--data_dir'], dest='data_dir', nargs=None, const=None, default='./data/', type=<class 'str'>, choices=None, help='base directory for saving.', metavar=None)

## Arguement Setup

In [3]:
args = parser.parse_args(
['--data', 'mnistvector', '--arch', 'layers50', '--samples', '1000']
)


## CUDA

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Model Directory

In [5]:
model_dir = os.path.join(args.save_dir, 
                         'forward',
                         f'{args.data}+{args.arch}',
                         f'samples{args.samples}'
                         f'{args.tail}')
os.makedirs(model_dir, exist_ok=True)
utils.save_params(model_dir, vars(args))
print(model_dir)

./saved_models/forward\mnistvector+layers50\samples1000


## Data

In [6]:
trainset, testset, num_classes = L.load_dataset(args.data, data_dir=args.data_dir)
X_train, y_train = F.get_samples(trainset, args.samples)
X_train, y_train = X_train.to(device), y_train.to(device)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9913344it [00:00, 11514874.47it/s]                             


Extracting ./data/MNIST\raw\train-images-idx3-ubyte.gz to ./data/MNIST\raw


29696it [00:00, 29804750.32it/s]         


Extracting ./data/MNIST\raw\train-labels-idx1-ubyte.gz to ./data/MNIST\raw




1649664it [00:00, 13345842.95it/s]                             
5120it [00:00, 5039858.36it/s]          

Extracting ./data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/MNIST\raw

Extracting ./data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/MNIST\raw



C:\anaconda\envs\python3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Architectur

In [7]:
net = L.load_architecture(args.data, args.arch)
net = net.to(device)

In [8]:
with torch.no_grad():
    Z_train = net.init(X_train, y_train)
    losses_train = net.get_loss()
    X_train, Z_train = F.to_cpu(X_train, Z_train)

0 | 85.450424 289.671265 204.220840
1 | 98.485947 301.172058 202.686111
2 | 113.187408 314.530212 201.342804
3 | 129.490341 329.732544 200.242203
4 | 147.266739 346.672516 199.405777
5 | 166.326157 365.155396 198.829239
6 | 186.423218 384.914154 198.490936
7 | 207.273956 405.637634 198.363678
8 | 228.578842 426.997131 198.418289
9 | 250.043884 448.670197 198.626312
10 | 271.398010 470.357239 198.959229
11 | 292.405685 491.794800 199.389114
12 | 312.865326 512.752136 199.886810
13 | 332.616425 533.039795 200.423370
14 | 351.541672 552.515625 200.973953
15 | 369.564331 571.082825 201.518494
16 | 386.648727 588.692017 202.043289
17 | 402.781631 605.319702 202.538071
18 | 417.974091 620.970764 202.996674
19 | 432.241547 635.656860 203.415314
20 | 445.605835 649.398132 203.792297
21 | 458.095764 662.223145 204.127380
22 | 469.743958 674.164551 204.420593
23 | 480.587021 685.259033 204.672012
24 | 490.666199 695.548828 204.882629
25 | 500.026672 705.080322 205.053650
26 | 508.721039 713.9078


## Saving

In [9]:
utils.save_loss(model_dir, 'train', losses_train)
utils.save_ckpt(model_dir, 'model', net)

In [10]:
plot.plot_loss_mcr(model_dir, 'train')

Plot saved to: ./saved_models/forward\mnistvector+layers50\samples1000\figures\loss_mcr\train.png
